## Image Classification

### 1.Firstly we will do the data preparation

In [14]:
pip install opencv-python

  Using cached https://files.pythonhosted.org/packages/85/17/bad54f67bbe27d88ba520c3f59315e95b4e254cd28767c20accacb0597d8/opencv_python-4.2.0.34-cp37-cp37m-win_amd64.whl
Note: you may need to restart the kernel to use updated packages.


In [31]:
import os,sys
import cv2

### cv2.imread()
cv2.imread() method loads an image from the specified file. If the image cannot be read (because of missing file, improper permissions, unsupported or invalid format) then this method returns an empty matrix.

Syntax: cv2.imread(path, flag)

#### Parameters:
path: A string representing the path of the image to be read.
flag: It specifies the way in which image should be read. It’s default value is cv2.IMREAD_COLOR

### cv2.imwrite()

OpenCV-Python is a library of Python bindings designed to solve computer vision problems

cv2.imwrite() method is used to save an image to any storage device. 
This will save the image according to the specified format in current working directory.

#### Syntax: cv2.imwrite(filename, image)

#### Parameters:
filename: A string representing the file name. The filename must include image format like .jpg, .png, etc.
image: It is the image that is to be saved.

In [46]:
def listfiles(rootDir):
  
      list_dirs = os.walk(rootDir)
      for root, dirs, files in list_dirs:
        #print(root,dirs,files)
        #for d in dirs:
            #print(os.path.join(root,d))
      
        for f in files:
            field = f.split('.')[0]
            
            #os.path.join() method in Python join one or more path components intelligently. 
            filepath = os.path.join(root,f)
            try:
                src = cv2.imread(filepath,1)
                print("src=",filepath,src.shape)
                os.remove(filepath)
                cv2.imwrite(os.path.join(root,field + ".jpg"),src)
            except:
                os.remove(filepath)
                continue

In [47]:
rootDir = 'C:\\Users\\zjx_n\\Desktop\\labs\\Image-Downloader\\download_images\\Beak'
listfiles(rootDir)

src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0000.jpg (640, 640, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0001.jpg (600, 562, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0002.jpg (852, 960, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0003.jpg (704, 900, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0004.jpg (595, 583, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0005.jpg (300, 300, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0006.jpg (1144, 750, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0009.jpg (602, 640, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0010.jpg (902, 640, 3)
src= C:\Users\zjx_n\Desktop\labs\Image-Downloader\download_images\Beak\Google_0011.jpg (24